# Diseño y Vlsm 

# 1. Diseño de Direccionamiento IP (VLSM) - Grupo 10
**Restricción Crítica:** Bloque Asignado `10.2.64.0/18` (Rango: `.64.0` a `.127.255`).

## 1.1 Criterios de Diseño
Para cumplir con los requisitos contradictorios se aplicó una estrategia de **Segmentación Jerárquica**:

1.  **Red Central (Río):** Se asignó el 50% del espacio total (`/19`) a la VLAN Administrativa para maximizar su capacidad (8,190 hosts), acercándose lo más posible al ideal de 10k sin bloquear el resto del proyecto.
2.  **Sedes Remotas:** Se aplicó la fórmula de clase:
    $$Base (1000) + (10 \times 11) = 1110 \text{ hosts}$$
    Esto requiere bloques de máscara **/21** (2,048 IPs) por sede.

## 1.2 Distribución de Bloques Mayores (CIDR)
| Sede | Rol | Máscara | Rango Asignado | Capacidad Real |
| :--- | :--- | :--- | :--- | :--- |
| **Río de Janeiro** | Sede Central | **/19** | **`10.2.64.0`** | 8,190 Hosts |
| **São Paulo** | Sede Remota | **/21** | **`10.2.96.0`** | 2,046 Hosts |
| **Salvador** | Sede Remota | **/21** | **`10.2.104.0`** | 2,046 Hosts |
| **Recife** | Sede Remota | **/21** | **`10.2.112.0`** | 2,046 Hosts |
| **Infraestructura** | WANs / Reserva | **/21** | **`10.2.120.0`** | *(Para Enlaces y Servers)* |

## 1.3 Tabla Detallada de Subredes (VLANs)

### 📍 Bloque RÍO (`10.2.64.0/19`)
| Función | VLAN | Red | Máscara | Rango Útil |
| :--- | :--- | :--- | :--- | :--- |
| **Admin (Masiva)** | 10 | `10.2.64.0` | **/19** | `.64.1` - `.95.254` |

### 📍 Bloque SÃO PAULO (`10.2.96.0/21`)
| Función | VLAN | Red | Máscara | Rango Útil |
| :--- | :--- | :--- | :--- | :--- |
| **Prensa** | 30 | `10.2.96.0` | **/23** | `.96.1` - `.97.254` |
| **Producción** | 40 | `10.2.98.0` | **/24** | `.98.1` - `.98.254` |
| **Wi-Fi Invitados**| 60 | `10.2.99.0` | **/24** | `.99.1` - `.99.254` |

### 📍 Bloque SALVADOR (`10.2.104.0/21`)
| Función | VLAN | Red | Máscara | Rango Útil |
| :--- | :--- | :--- | :--- | :--- |
| **Logística** | 20 | `10.2.104.0` | **/23** | `.104.1` - `.105.254`|
| **IoT** | 50 | `10.2.106.0` | **/24** | `.106.1` - `.106.254`|
| **Wi-Fi Invitados**| 60 | `10.2.107.0` | **/24** | `.107.1` - `.107.254`|

### 📍 Bloque RECIFE (`10.2.112.0/21`)
| Función | VLAN | Red | Máscara | Rango Útil |
| :--- | :--- | :--- | :--- | :--- |
| **Seguridad** | 40 | `10.2.112.0` | **/23** | `.112.1` - `.113.254`|
| **Soporte** | 99 | `10.2.114.0` | **/24** | `.114.1` - `.114.254`|
| **Wi-Fi Invitados**| 60 | `10.2.115.0` | **/24** | `.115.1` - `.115.254`|

### 📍 Bloque INFRAESTRUCTURA (`10.2.120.0/21`)
*(Reservado para servicios críticos y WANs)*
| Función | Red | Máscara | Notas |
| :--- | :--- | :--- | :--- |
| **Enlaces WAN** | `10.2.120.0` | **/24** | Subneteo /30 (Ver Tabla WAN) |
| **Servidores Río** | `10.2.121.0` | **/24** | `.121.1` - `.121.254` |
| **IoT Río** | `10.2.122.0` | **/24** | `.122.1` - `.122.254` |
| **Seguridad Río** | `10.2.123.0` | **/24** | `.123.1` - `.123.254` |
| **Cable Admin** | `10.2.124.0` | **/30** | Gestión Fuera de Banda |